# EXERCISE 3.23
3.22を整数だったり加算だったりに限定しないものを作る。

型を一般化して、関数を定義できるようにするってことか。

In [2]:
sealed trait List[+A]
case object Nil extends List[Nothing]
case class Cons[+A](head: A, tail: List[A]) extends List[A]

object List {
    def sum(ints: List[Int]): Int =
        ints match {
            case Nil => 0
            case Cons(x, xs) => x + sum(xs)
        }
    
    def apply[A](as: A*): List[A] =
        if (as.isEmpty) Nil
        else Cons(as.head, apply(as.tail: _*))
    
    def tail[A](as: List[A]): List[A] =
        as match {
            case Nil => Nil
            case Cons(h, t) => t
        }
    
    def setHead[A](l: List[A], a: A): List[A] =
        l match {
            case Nil => apply(a)
            case Cons(_, t) => Cons(a, t)
        }
    
    def drop[A](l: List[A], n: Int): List[A] =
        if (n <= 0) l
        else l match {
            case Nil => Nil
            case Cons(_, t) => drop(t, n-1)
        }
    
    def dropWhile[A](l: List[A], f: A => Boolean): List[A] =
        l match {
            case Cons(h, t) => if (f(h)) dropWhile(t, f) else l
            case _ => l
        }
    
    def init[A](l: List[A]): List[A] =
        l match {
            case Nil => sys.error("empty")
            case Cons(_,Nil) => Nil
            case Cons(h,t) => Cons(h, init(t))
        }
 
    def foldRight[A,B](as: List[A], z: B)(f: (A, B) => B): B = // Utility functions
        as match {
          case Nil => z
          case Cons(x, xs) => f(x, foldRight(xs, z)(f))
    }
    
    def length[A](as: List[A]): Int =
        foldRight(as, 0)((x,y) => 1 + y)
    
    def append[A](a1: List[A], a2: List[A]): List[A] =
        a1 match {
          case Nil => a2
          case Cons(h,t) => Cons(h, append(t, a2))
    }

    
    @annotation.tailrec
    def foldLeft[A,B](as: List[A], z: B)(f: (B, A) => B): B =
        as match {
            case Nil => z
            case Cons(x, xs) => foldLeft(xs, f(z, x))(f)
        }
    
    def reverse[A](as: List[A]): List[A] =
        foldLeft(as, Nil:List[A])((z, l) => Cons(l, z))
    
    def foldRightViaFoldLeft[A,B](l: List[A], z: B)(f: (A,B) => B): B =
        foldLeft(reverse(l), z)((b,a) => f(a,b))

    def foldRightViaFoldLeft_1[A,B](l: List[A], z: B)(f: (A,B) => B): B =
        foldLeft(l, (b:B) => b)((g,a) => b => g(f(a,b)))(z)

    def foldLeftViaFoldRight[A,B](l: List[A], z: B)(f: (B,A) => B): B =
        foldRight(l, (b:B) => b)((a,g) => b => g(f(b,a)))(z)
    
    def concat[A](l: List[List[A]]): List[A] =
        foldRight(l, Nil:List[A])(append(_,_))
    
    def map[A,B](as: List[A])(f: A=>B): List[B] =
        foldRight(as, Nil:List[B])((h,t) => Cons(f(h), t))
    
    def filter[A](as: List[A])(f: A => Boolean): List[A] =
        foldRight(as, Nil:List[A])((h,t) => if (f(h)) Cons(h,t) else t) // foldRightViaFoldLeft is better
    
    def flatMap[A,B](as: List[A])(f: A => List[B]): List[B] =
        //concat(map(as)(f))
        foldRight(as, Nil:List[B])((h,t) => append(f(h), t))
    
    def addPairwise(l: List[Int], r: List[Int]): List[Int] =
        (l, r) match {
            case (Nil, _) => Nil
            case (_, Nil) => Nil
            case (Cons(hl, tl), Cons(hr, tr)) => Cons(hl + hr, addPairwise(tl, tr))
        }
    
    def zipWith[A,B,C](l: List[A], r: List[B])(f: (A,B)=>C): List[C] = // may called zipWith
        (l, r) match {
            case (Nil, _) => Nil
            case (_, Nil) => Nil
            case (Cons(hl, tl), Cons(hr, tr)) => Cons(f(hl, hr), zipWith(tl, tr)(f))
        }
}

List.zipWith(List(1,2,3), List(4,5,6))(_+_)

defined trait List
defined object Nil
defined class Cons
defined object List
res0_4: $user.List[Int] = Cons(5,Cons(7,Cons(9,Nil)))

受けるリストを同じにする必要がない、というのは気が付かなかったので、答え見て修正。
まぁ、そうかも。